In [190]:
!pip install optuna -q

     |████████████████████████████████| 184kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 41.1MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 6.3MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 8.3MB/s  eta 0:00:01
     |████████████████████████████████| 51kB 6.5MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 7.3MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 56.0MB/s eta 0:00:01


In [191]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

import optuna

In [168]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
X_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values


X_test = pd.read_csv('../data/Xte.csv',sep=',',index_col=0).values
X_train = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0).values

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0).values

In [169]:
print('x_train: {} y_train {}'.format(X_train_mat100.shape,y_train.shape))
print('x_train: {} y_train {}'.format(X_train.shape,y_train.shape))
print('x_test: {}'.format(X_test.shape))

x_train: (2000, 100) y_train (1800, 1)
x_train: (2000, 1) y_train (1800, 1)
x_test: (1000, 1)


In [176]:
X_train_mat100 = scale(X_train_mat100)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_mat100, y, test_size=0.33, random_state=42)

print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)

(1340, 100) (1000, 1) (1340, 1) (200, 1)


### Model Test

In [171]:
from sklearn.metrics import  accuracy_score
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        hassien = np.dot(y_pred.T,(1-y_pred))*np.linalg.inv(np.dot(x.T,x))
        return np.dot(hassien,np.dot(x.T,(y_pred-y)))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra #+ np.sum(self.params)/1
            
            if i%10 == 0 or i == self.epoch-1:
                print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [187]:
class logisticregression():
    def __init__(self,train_data,train_labels,lr=0.01,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        return np.dot(x.T,(y_pred-y))
    
    def train(self):
        for i in range(self.epoch):
            y_pred = self.sigmoid(np.dot(self.train_data,self.params))
            loss = self.cost(self.train_labels,y_pred)
            
            gra = self.gradient(self.train_labels,y_pred,self.train_data)
            
            self.params -= self.lr*gra
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result >= 0.5 ] = 1
        result[result < 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [188]:
def cross_validate(x_data,y_data,lr,k=5):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
            
        logistic = logisticregression(x_train,y_train,lr=lr,epoch=10,print_every=None)
        logistic.train()
        
        result = logistic.evaluate(x_test,y_test)
        aggrigate_result.append(result)
        
    return sum(aggrigate_result)/len(aggrigate_result)

In [193]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    k =  trial.suggest_int('k', 2, 10,2)
    return cross_validate(X_train_mat100, y,lr,k)

In [200]:
# cross_validate(X_train_mat100, y,0.001,10)

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=10,show_progress_bar=True)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2020-05-23 14:42:36,783] Finished trial#0 with value: 0.5695 with parameters: {'lr': 0.0013803755945249333, 'k': 8}. Best is trial#0 with value: 0.5695.
[I 2020-05-23 14:42:36,894] Finished trial#1 with value: 0.5675 with parameters: {'lr': 0.002694734190662389, 'k': 8}. Best is trial#0 with value: 0.5695.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning:

invalid value encountered in multiply



[I 2020-05-23 14:42:37,053] Finished trial#2 with value: 0.517 with parameters: {'lr': 0.030849098801601697, 'k': 10}. Best is trial#0 with value: 0.5695.
[I 2020-05-23 14:42:37,155] Finished trial#3 with value: 0.55 with parameters: {'lr': 0.05356936408037155, 'k': 4}. Best is trial#0 with value: 0.5695.
[I 2020-05-23 14:42:37,257] Finished trial#4 with value: 0.582 with parameters: {'lr': 0.0003072771139391344, 'k': 4}. Best is trial#4 with value: 0.582.
cant vsplit 2000  by  6
[W 2020-05-23 14:42:37,332] Setting status of trial#5 as TrialState.FAIL because the returned value from the objective function cannot be casted to float. Returned value is: None
[I 2020-05-23 14:42:37,443] Finished trial#6 with value: 0.516 with parameters: {'lr': 0.00883605103054329, 'k': 10}. Best is trial#4 with value: 0.582.
[I 2020-05-23 14:42:37,569] Finished trial#7 with value: 0.5395000000000001 with parameters: {'lr': 0.0043516166100189225, 'k': 8}. Best is trial#4 with value: 0.582.
[I 2020-05-23 14

In [201]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
# df = df.dropna()
df.sort_values(by=['value'])

,number,value,duration,params_k,params_lr,system_attrs_fail_reason
6,6,0.5160,00:00:00.109123,10,0.008836,NaN
2,2,0.5170,00:00:00.153537,10,0.030849,NaN
7,7,0.5395,00:00:00.119214,8,0.004352,NaN
3,3,0.5500,00:00:00.098511,4,0.053569,NaN
1,1,0.5675,00:00:00.106884,8,0.002695,NaN
0,0,0.5695,00:00:00.155160,8,0.001380,NaN
8,8,0.5785,00:00:00.107135,4,0.000486,NaN
4,4,0.5820,00:00:00.097582,4,0.000307,NaN
5,5,NaN,00:00:00.070165,6,0.004174,Setting status of trial#5 as TrialState.FAIL b...
9,9,NaN,00:00:00.067149,6,0.000114,Setting status of trial#9 as TrialState.FAIL b...


In [202]:
# X_train, X_test, y_train, y_test
logistic = logisticregression(X_train,y_train,epoch=5,print_every=1,lr=0.000307)
logistic.train()
        
result = logistic.evaluate(X_val,y_val)
result

Epoch : 0  Loss: 0.6931471805599453
Epoch : 1  Loss: 0.6710256163735592
Epoch : 2  Loss: 0.6569848839682912
Epoch : 3  Loss: 0.6471494677918743
Epoch : 4  Loss: 0.6399666752924154


0.6015151515151516

In [203]:
sumbission = []
for i in range(len(X_test_mat100)):
    result = logistic.predict(X_test_mat100[i,:])
    sumbission.append([i,int(result)])
    

In [204]:
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('test.csv',index=False)